In [3]:
import datetime

import requests
from bs4 import BeautifulSoup as bs

---

### base code


In [ ]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","").replace("조","").replace(",","")

def getName(soup):
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cn=&cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

def getJsonFromConsen(code):
    consen_url = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&finGubun=MAIN&frq=0"+\
        "&chartType=svg&cmp_cd="
    return requests.get(consen_url + code).json()    

---

### 주식 이름 판단 

In [8]:
def isETF(soup):
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

def isETN(soup):
    name = getName(soup)
    if("ETN" in name):
        return True
    return False

def isFirst(soup):
    name = getName(soup)
    if(name[-1] == "우"):
        return True
    if(name[-2:] in ["우A", "우B", "우C"]):
        return True
    return False

def isStockByName(name):
    if("ETF" in name):
        return False
    if("ETN" in name):
        return False
    if(name[-1] == "우"):
        return False
    if(name[-2:] in ["우A", "우B", "우C"]):
        return False
    return True

---

### 시가총액 3분위, 10분위 기준 구하기

In [9]:
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="
sizeL = []
soup = bs(requests.get(size_market_url_0+"1").text, 'html.parser')

In [10]:
for i in range(1, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.select("td.number")[4].text for x in soup.select("table")[1].select_one("tbody").findAll("tr",{"onmouseout":"mouseOut(this)"}) if isStockByName(x.select_one("a.tltle").text)]
    sizeL += [int(x.replace(",","")) for x in tempL]
for i in range(1, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.select("td.number")[4].text for x in soup.select("table")[1].select_one("tbody").findAll("tr",{"onmouseout":"mouseOut(this)"}) if isStockByName(x.select_one("a.tltle").text)]
    sizeL += [int(x.replace(",","")) for x in tempL]


In [11]:
sizeL = sorted(sizeL)

# 3분위까지의 기준
size3 = sizeL[:int(265.6*3)][-1]
print(size3)

# 10분위 까지의 기준
size10 = sizeL[-265:][0]
print(size10)

541
8022


---

### 시가총액 목록 사용하기

In [13]:
# 시가총액 300억 이하 목록 구하기
codeL = []

# 코스피 300억 이하 (후보)
for i in range(1, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
for i in range(1, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    
# 시가총액 작은 애들부터 보기
codeL.reverse()

---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


In [15]:
def perUnder7(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float < 7 and per_float > 0):
            return per_float
        return -1
    except:
        return -1
    
def pbrUnder1(soup_report):
    try:
        pbr_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
        pbr_float = float(cleanText(pbr_str))
        if(pbr_float < 1 and pbr_float > 0):
            return pbr_float
        return -1
    except:
        return -1

def per_avg(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float > 0):
            return per_float
        return -1
    except:
        return -1

def marketSumBad(soup):
    marketSum = int(cleanText(soup.select_one("#_market_sum").text))
    if(marketSum <= size3 or marketSum >= size10):
        return False
    return True

---

## 괜찮은 종목 거르기

In [17]:
good = []
for code in codeL:
    
    # 자원 확보
    soup = getSoupFromCode(code)
    soup_report = getSoupFromReport(code)
    name = getName(soup)
    
    # etf/etn/우선주/300초과 제외
    if(isETF(soup) or isETN(soup) or isFirst(soup) or marketSumBad(soup)):
        continue
        
    # 0<per<7, 0<pbr<1
    per = perUnder7(soup_report)
    pbr = pbrUnder1(soup_report)
    perAvg = per_avg(soup_report)
    if(per == -1 or pbr
       == -1 or perAvg == -1):
        continue
        
    # 출력
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

에스앤씨엔진그룹 4.41 0.06
영신금속 5.17 0.62
동일철강 6.06 0.46
패션플랫폼 6.01 0.75
골든센츄리 1.32 0.16
한국컴퓨터 4.29 0.39
원일특강 4.29 0.33
오가닉티코스메틱 1.46 0.14
케이디켐 6.38 0.59
메디앙스 4.83 0.67
신원종합개발 4.24 0.58
푸드웰 1.69 0.64
세원물산 4.48 0.27
코리아에셋투자증권 6.71 0.74
에스에이티 5.74 0.62


KeyboardInterrupt: 

In [22]:
len(good)

11

In [23]:
good

['성우테크론 6.63 0.57',
 '신원종합개발 2.38 0.35',
 '원일특강 3.79 0.32',
 '한국컴퓨터 4.94 0.38',
 '바이오로그디바이스 6.52 0.73',
 '코리아에셋투자증권 5.4 0.62',
 '케이디켐 6.06 0.57',
 '한솔로지스틱스 6.16 0.77',
 '진양산업 5.53 0.92',
 'SJM 5.79 0.24',
 '동원수산 6.55 0.78']